In [4]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://ppa

In [6]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-03-29 02:52:50--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.75MB/s    in 0.2s    

2021-03-29 02:52:51 (5.75 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [48]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
#url = "https://s3.console.aws.amazon.com/s3/object/lc-1st-s3-bucket?region=us-east-1&prefix=vine_table.csv"

spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

In [49]:
df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'string'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'string')]

In [50]:
# Filter by price on certain columns
vote_df=df.filter(df["total_votes"]>=20)
vote_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   44689470|R2WOW0TURNXB26|B00YY3UBV2|     310491927|            Garmin 1|        Wireless|          3|           54|         59|   N|                Y|Pretty Disappoint...|Bought this unit ...| 2015-08-31|
|         US|     112342|R13VL62Y2HBQ0B|B010VFZJD6|     129632031|iTaste MVP3 PRO -...|        Wireless|          5|    

In [51]:
vote_df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'string'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'string')]

In [52]:
# retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%
helpful_vote_df=vote_df.filter(vote_df["helpful_votes"]/vote_df["total_votes"]>=0.5)
helpful_vote_df.show(5)
#helpful_vote_df=vote_df.filter()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   44689470|R2WOW0TURNXB26|B00YY3UBV2|     310491927|            Garmin 1|        Wireless|          3|           54|         59|   N|                Y|Pretty Disappoint...|Bought this unit ...| 2015-08-31|
|         US|     112342|R13VL62Y2HBQ0B|B010VFZJD6|     129632031|iTaste MVP3 PRO -...|        Wireless|          5|    

In [53]:
# Filter on Vine==N
n_vine_df=helpful_vote_df.filter(helpful_vote_df["vine"] == "N")
n_vine_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   44689470|R2WOW0TURNXB26|B00YY3UBV2|     310491927|            Garmin 1|        Wireless|          3|           54|         59|   N|                Y|Pretty Disappoint...|Bought this unit ...| 2015-08-31|
|         US|     112342|R13VL62Y2HBQ0B|B010VFZJD6|     129632031|iTaste MVP3 PRO -...|        Wireless|          5|    

In [56]:
# Filter on Vine==Y
y_vine_df=helpful_vote_df.filter(helpful_vote_df["vine"] == "Y")
y_vine_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48852155|R1MAOLI5FJHAFM|B013X0V11K|     610966690|BLU Studio 7.0 ll...|        Wireless|          4|          249|        261|   Y|                N|More than just a ...|Ever since the ve...| 2015-08-31|
|         US|   11556116| R9PYAUDIBJVEC|B013X0V4VM|     672788343|BLU Studio C Supe...|        Wireless|          4|    

In [57]:
#Calc the total number of vine reviews
y_vine_total=y_vine_df.groupby("vine").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "review")
y_vine_total.show()

+----+------+
|vine|review|
+----+------+
|   Y|   613|
+----+------+



In [75]:
#Calc the total number of non-vine reviews
n_vine_total=n_vine_df.groupby("vine").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "review")
n_vine_total.show()

+----+------+
|vine|review|
+----+------+
|   N| 64968|
+----+------+



In [ ]:
n_vine_total.dtypes

In [82]:
#Create df for 5 star vine reviews
y_5_vine_total=y_vine_df.filter(y_vine_df["star_rating"]=="5")
y_5_vine_total.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   46671309| R6V9SHMMG5M8F|B013X0V11K|     610966690|BLU Studio 7.0 ll...|        Wireless|          5|          101|        110|   Y|                N|  Very good Phablet!|I remember the da...| 2015-08-30|
|         US|   53019847|R3SRW1E8J56IGV|B0129TQLPW|     226174255|BLU Energy X Plus...|        Wireless|          5|    

In [61]:
#Create df for 5 star non-vine reviews
n_5_vine_total=n_vine_df.filter(n_vine_df["star_rating"]=="5")
n_5_vine_total.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|     112342|R13VL62Y2HBQ0B|B010VFZJD6|     129632031|iTaste MVP3 PRO -...|        Wireless|          5|           15|         21|   N|                Y|   Excellent battery|I love my Mvp pro...| 2015-08-31|
|         US|   22953177|R111DJA10Y6CMU|B013BHLU66|     396362963|Waterproof Case f...|        Wireless|          5|    

In [70]:
#Calc the total number of 5 star vine reviews
y_5_vine_total_sum=y_5_vine_total.groupby("vine").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "5_star_review")
y_5_vine_total_sum.show()

+----+-------------+
|vine|5_star_review|
+----+-------------+
|   Y|          222|
+----+-------------+



In [81]:
vine_df=vine_df.withColumn("star_rating",vine_df["star_rating"].cast('Integer'))

y_5_vine_total_sum.dtypes

[('vine', 'string'), ('5_star_review', 'bigint')]

In [89]:
print(y_vine_total["review"][0])

Column<b'review[0]'>


In [90]:
#Add percentage of 5 star view for Vine Review
#y_5_vine_total_sum=y_5_vine_total_sum.withColumn("% of 5 Star Review",(y_5_vine_total_sum["5_star_review"]/y_vine_total["review"]))
y_5_vine_total_sum=y_5_vine_total_sum.withColumn("% of 5 Star Review",(y_5_vine_total_sum["5_star_review"]/613*100))
y_5_vine_total_sum.show()

+----+-------------+------------------+
|vine|5_star_review|% of 5 Star Review|
+----+-------------+------------------+
|   Y|          222|36.215334420880914|
+----+-------------+------------------+



In [66]:
#Calc the total number of 5 star non-vine reviews
n_5_vine_total_sum=n_5_vine_total.groupby("vine").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "5_star_review")
n_5_vine_total_sum.show()

+----+-------------+
|vine|5_star_review|
+----+-------------+
|   N|        30543|
+----+-------------+



In [91]:
#Add percentage of 5 star view for non-Vine Review
n_5_vine_total_sum=n_5_vine_total_sum.withColumn("% of 5 Star Review",(n_5_vine_total_sum["5_star_review"]/64968*100))
n_5_vine_total_sum.show()

+----+-------------+------------------+
|vine|5_star_review|% of 5 Star Review|
+----+-------------+------------------+
|   N|        30543| 47.01237532323606|
+----+-------------+------------------+

